In [1]:
import numpy as np
import pyvista as pv
import os

In [9]:
sim=70
pref = 'SP'
data = np.load('postprocessed_data/'+pref+'_data/data.npy')
coords = np.load('postprocessed_data/'+pref+'_data/coords.npy')
coords_o = np.load('postprocessed_data/'+pref+'_data/coords_o.npy')
sdf = np.load('postprocessed_data/'+pref+'_data/sdf.npy')
sdf_o = np.load('postprocessed_data/'+pref+'_data/sdf_o.npy')
sdf_o = sdf_o[sim,:,:]

tidx = -1
data_o = data[tidx,:,:] # original

# Geometry
mesh_path = '/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/SP/SP_'+str(sim).zfill(4)+'/'
filepath = mesh_path + '4-procs/result_150.vtu'
mesh = pv.read(filepath)
boundary = mesh.decimate_boundary().extract_all_edges()

# Reshape for StructuredGrid
mesh_o = pv.StructuredGrid()
mesh_o.points = coords_o
mesh_o.dimensions = (100, 20, 20)  

# Add scalar field
mesh_o['u'] = data_o[:,0]
mesh_o['v'] = data_o[:,1]
mesh_o['w'] = data_o[:,2]
mesh_o['p'] = data_o[:,3]
# mesh_o['p'] = mesh_o['p']/np.maximum(mesh_o['p'],5e4)
mesh_o['g'] = sdf_o
mesh_o['m'] = sdf_o>0
mesh_o['walls'] = sdf_o<0#np.logical_and(sdf_o>-0.05, sdf_o < 0.05)

# Create sampling box
bounds = [1, 3.5, -0., 0.75, -0.75, 0.75]
# bounds = [0.4,0.6,0.3,0.7,0.3,0.7]

# Extract coordinates of all points in the mesh
points = mesh_o.points

# Create a mask for points inside the box
mask = (
    (points[:, 0] >= bounds[0]) & (points[:, 0] <= bounds[1]) &
    (points[:, 1] >= bounds[2]) & (points[:, 1] <= bounds[3]) &
    (points[:, 2] >= bounds[4]) & (points[:, 2] <= bounds[5]) #|
    # mesh_o.point_data['walls'] 
)

# Get the indices of the points within the box
ids = np.where(mask)[0]

# Extract point data and create a new point cloud mesh
# sampled_data = mesh_o.extract_points(ids)
# sample_x = mesh_o.points[mask]
# sample_u = mesh_o['u'][mask]
# sample_v = mesh_o['v'][mask]
# sample_w = mesh_o['w'][mask]
# sample_p = mesh_o['p'][mask]
# sample_data = pv.PolyData(sample_x)
# sample_data['u'] = sample_u
# sample_data['v'] = sample_v
# sample_data['w'] = sample_w
# sample_data['p'] = sample_p
sample_data = mesh_o.extract_points(mask, adjacent_cells=False)

# Normalize
# for scalar in mesh_o.array_names:
#     if scalar != 'g' or scalar !='m':
#         mesh_o[scalar] = mesh_o[scalar]/np.max(np.abs(mesh_o[scalar]))#*mesh_o['m'] 

# Define scalar and value ranges
scalar = 'u'
clim_dict = {
    'u': (-1,1),
    'v': (-1,1),
    'w': (-1,1),
    'p': (-1,1),
    'g': (-1,1),
    'm': (0,1),
}

# # Define the start and end points of line
# start_point = [-4, -2, 0]
# end_point = [4, 2, 0]

# # Sample the data from the dataset onto the line
# sampled_data = mesh_o.sample_over_line(start_point,end_point,resolution=20)

# Clip for vis
mesh_o = mesh_o.clip('y')

# Visualize
pl = pv.Plotter(off_screen=True,window_size=(800,400))
pl.add_mesh(boundary,opacity=0.3)
# pl.add_mesh(mesh_o, scalars=scalar,
#             # clim=clim_dict[scalar],
#             # opacity=0.3
#             # show_edges=True
#             )
# n_samples = 10
# random_ids = np.random.choice(sample_data.number_of_cells, n_samples, replace=False)
# Extract those cells
# sample_data = sample_data.extract_cells(random_ids)
pl.add_mesh(sample_data,
            # clim=()
            scalars=scalar,
            cmap='viridis',
            show_scalar_bar=False)#, style='points', point_size=10)
pl.enable_parallel_projection()
pl.camera_position='xz'
pl.camera.zoom(2)
pl.screenshot('/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/images/SP_doppler.png')
pl.show()


Widget(value='<iframe src="http://localhost:39729/index.html?ui=P_0x7cc567f6d7c0_6&reconnect=auto" class="pyvi…

In [3]:
# Load data
sim = 0
data = np.load('postprocessed_data/NP_data/data.npy')
coords_o = np.load('postprocessed_data/NP_data/coords_o.npy')
sdf_o = np.load('postprocessed_data/NP_data/sdf_o.npy')
sdf_o = sdf_o[sim,:,:]
Nt = 40

# save_path = 'postprocessed_data/NP_data/doppler/sim'
save_path = '/home/dana/Documents/CoNFiLD/CoNFiLD/NoisyPipe/data/doppler/'

sample_u_arr = []
sample_v_arr = []
sample_w_arr = []
sample_p_arr = []
for i in range(Nt):
    data_o = data[i,:,:] # original

    # Reshape for StructuredGrid
    mesh_o = pv.StructuredGrid()
    mesh_o.points = coords
    mesh_o.dimensions = (100, 20, 20)  

    # Add scalar field
    mesh_o['u'] = data_o[:,0]
    mesh_o['v'] = data_o[:,1]
    mesh_o['w'] = data_o[:,2]
    mesh_o['p'] = data_o[:,3]
    mesh_o['g'] = sdf_o
    mesh_o['m'] = sdf_o>0
    mesh_o['walls'] = sdf_o<0#np.logical_and(sdf_o>-0.05, sdf_o < 0.05)

    # Create sampling box
    bounds = [-1, 1, -0.5, 0.5, -0.5, 0.5]
    # bounds = [0.4,0.6,0.3,0.7,0.3,0.7]

    # Extract coordinates of all points in the mesh
    points = mesh_o.points

    # Create a mask for points inside the box
    mask = (
        # (points[:, 0] >= bounds[0]) & (points[:, 0] <= bounds[1]) &
        # (points[:, 1] >= bounds[2]) & (points[:, 1] <= bounds[3]) &
        # (points[:, 2] >= bounds[4]) & (points[:, 2] <= bounds[5]) 
        mesh_o.point_data['walls'] 
    )

    # Extract point data and create a new point cloud mesh
    # sampled_data = mesh_o.extract_points(ids)
    sample_u_arr.append(mesh_o['u'][mask])
    sample_v_arr.append(mesh_o['v'][mask])
    sample_w_arr.append(mesh_o['w'][mask])
    sample_p_arr.append(mesh_o['p'][mask])

    
# Get the indices of the points within the box
ids = np.where(mask)[0]

# Build arrays
sample_x = np.array(mesh_o.points[mask])
sample_g = np.expand_dims(np.array(mesh_o['g'][mask]),1)
sample_u = np.stack(sample_u_arr)
sample_v = np.stack(sample_v_arr)
sample_w = np.stack(sample_w_arr)
sample_p = np.stack(sample_p_arr)

# Save
np.save(save_path + 'sim' + str(sim) + '_x.npy',sample_x)
np.save(save_path + 'sim' + str(sim) + '_u.npy',sample_u)
np.save(save_path + 'sim' + str(sim) + '_v.npy',sample_v)
np.save(save_path + 'sim' + str(sim) + '_w.npy',sample_w)
np.save(save_path + 'sim' + str(sim) + '_p.npy',sample_p)
np.save(save_path + 'sim' + str(sim) + '_sdf.npy',sample_g)
np.save(save_path + 'sim' + str(sim) + '_ids.npy',ids)

In [78]:
np.max(sample_u)
sample_u.shape

(40, 25500)